# Capstone project:  The Battle of Neighborhoods

#### Babloo Kumar Sah
Data scientist | Data Analysis | Data Visulalization 
Machine Learining | Prediction Modelling
Python | R | SQL

This notebook consist of python code that will use machine leaning algorithm to determine the likelyhood of new neighborhood based on facilities available around. We will use Foursquare API to obtain location dataset and determine the similarity of new polace in comparision to the old neighborhood.

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed
Libraries imported.


In [3]:
# importing scikit libraries for clustering

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [4]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [5]:
# we will define another function to extarct data from url and convert it into appropriate data frame.
def PreProcessUrlToDf(url):
    Neighborhood_details=requests.get(url).text
    
    # Now we will define a loop to extract information in an dataframe and rename column name to have meaninful name 
    linewise=Neighborhood_details.splitlines()
    df_url=pd.DataFrame(word.split(",") for word in linewise)
    df_url.rename(columns={0:'Neighborhood',
                       1:'Borough',
                       2:'City'},
            inplace=True)
    
    # removinmg unwated rows 
    df_url.drop(df_url.index[0], axis=0,inplace=True)
    
    #return the clenead and processed dataframe for further analysis.
    return(df_url)

In [6]:
# Function dcefination to strip "S.O" and "B.O" from neighborhood names. it will return an array of fromated neighborhood names
def FormatNeighName(df_format):
    Neighborhood=np.asarray(df_format['Neighborhood'])
    City=np.asarray(df_format['City'])
    Borough=np.asarray(df_format['Borough'])
    formatted_Neighborhood=[]
    formatted_city=[]
    formatted_Borough=[]
    
    for cityname,Neigh,Area in zip(City,Neighborhood,Borough):
        splitted_word=Neigh.split()
        Comb_Neighborhood=''
        
        stripped_cityname=cityname.replace('"','')
        stripped_Areaname=Area.replace('"','')
        
        for word in splitted_word:
            word=word.replace('"','')
            
            if word not in ["S.O","B.O"]:
                Comb_Neighborhood= Comb_Neighborhood + str(' ') + word
                
        formatted_Neighborhood.append(Comb_Neighborhood)
        formatted_city.append(stripped_cityname)
        formatted_Borough.append(stripped_Areaname)
        
    df_format['Neighborhood']=formatted_Neighborhood
    df_format['City']=formatted_city
    df_format['Borough']=formatted_Borough
    
    return(df_format)

In [32]:
# this function will merge the all the neighborhood and taluka name for each unique pincode and will present unique pincode only in dataframe.
def MergeNeighByTaluka(df_merge):
    
    column_index_neigh='Neighborhood' #will be used for indexing based on row nad column index name
    prev_Borough=''
    df_merge.sort_values("Borough",inplace=True)
    Neighborhood_array= np.asarray(df_merge['Neighborhood'])
    prev_index = 0
    Neigh_cur=''
    
    for df_index,Borough in zip(df_merge.index,df_merge['Borough']):
         
        if Borough != prev_Borough: #to skip pincode that are processed earlier
            df_merge.loc[prev_index,column_index_neigh]=Neigh_cur  #assigning merged values to original dataframe
            prev_Borough = Borough
            Neigh_cur=''
            prev_index = df_index
            
        if Neigh_cur == '':
            Neigh_cur=np.asarray(df_merge['Neighborhood'])[df_index]
        else:
            Neigh_cur= Neigh_cur + ',' + np.asarray(df_merge['Neighborhood'])[df_index]
 
    # grouping the dataframe by pincode to obatain data frame with unique pincode only and removinf the duplicate pincode value.
    df_processed=df_merge.groupby(['Borough'],as_index=False).agg({'City': 'first','Neighborhood': 'first'})
    return(df_processed)


In [48]:
#function to extract the lattitude and longitutude of given pincode and assign it to dataframe.
def ExtractLatLong(df_loc):
    
    #initializing a list to store latitutea nda longitute value for pincodes
    latitude=[]
    longitude=[]
    
    for index,Borough in enumerate(df_loc['Borough']):
     
        address= Borough + ',' + df_loc.loc[index,'City']
        geolocater=Nominatim(user_agent="my-application")
        location=geolocater.geocode(address)
        try:
            latitude.append(location.latitude)
            longitude.append(location.longitude)
        except:
            latitude.append('NA')
            longitude.append('NA')


    #inserting the new latitude and longitude colums in dataframe.
    df_loc = df_loc.assign(Latitude=latitude,Longitude=longitude)
    #df_loc[['Latitude','Longitude']] = pd.DataFrame([[np.asarray(latitude),np.asarray(longitude)]])
 
    return(df_loc)

In [22]:
# we take below two details as input form user 
# 1. Current neighborhood pincode 

cur_neighborhood_Name=input("Enter the current city taluka name:")
print("currentcity taluka name: is:" + str(cur_neighborhood_Name))

# 2. name of relocating city

Relocating_city=input("Enter the relocating city taluka name:")
print('Relocating city taluka name is :'+ str(Relocating_city))

Enter the current city taluka name:Pune
currentcity taluka name: is:Pune
Enter the relocating city taluka name:Mumbai
Relocating city taluka name is :Mumbai


#### We will  goverment databse API to extract the neighborhood, pincode and taluka name 

In [23]:
#To create api link
filter_field = '[districtname]'
filter_value = cur_neighborhood_Name

In [24]:
# The code was removed by Watson Studio for sharing.

'https://api.data.gov.in/resource/04cbe4b1-2f2b-4c39-a1d5-1c2e28bc0e32?api-key=579b464db66ec23bdd000001df0247bd5b8a438454e4cd76a7fd7787&filters[districtname]=Pune&fields=officename,taluk,districtname&format=csv&offset=0&limit=1000'

In [25]:
# we will call PreProcessUrlToDf that will extract required information from api and create a dataframe 
df_Cur_url=PreProcessUrlToDf(api_link)
df_Cur_url.head(5)

,Neighborhood,Borough,City
1,"""9 DRD B.O""","""Pune City""",Pune
2,"""AFMC S.O""","""Pune City""",Pune
3,"""Airport S.O (Pune)""","""Pune City""",Pune
4,"""Akurdi S.O""","""Pune City""",Pune
5,"""Ammunition Factory Khadki S.O""","""Pune City""",Pune


In [26]:
#we will call FormatNeighName that will remove unwanted charcter from neighbor hood as seen above like S.O" or B.O"
df_prepros_Neigh=FormatNeighName(df_Cur_url)
df_prepros_Neigh.head()

,Neighborhood,Borough,City
1,9 DRD,Pune City,Pune
2,AFMC,Pune City,Pune
3,Airport (Pune),Pune City,Pune
4,Akurdi,Pune City,Pune
5,Ammunition Factory Khadki,Pune City,Pune


In [42]:
#This will merge all the neighborhood and taluka names that belongs to same pincode. 
df_neighborhood_grouped=MergeNeighByTaluka(df_prepros_Neigh)
print(df_neighborhood_grouped.shape)
df_neighborhood_grouped.head()

(20, 3)


,Borough,City,Neighborhood
0,Ambegaon,Pune,"Chinchwadgaon, Ballalwadi, Sadalgaon, Varude,..."
1,Baramati,Pune,"Aundh Camp, Kudje, Pimpri, Dhayari, Lawale, J..."
2,Bhor,Pune,"Sarole, Delwadi, Sangamner, Kanhersar, Nazare..."
3,Daund,Pune,"Dapodi Bazar, Manjari BK, Khanote, Salisbury ..."
4,Haveli,Pune,"Chambli, Ordanance Dep Talegaon Dabhade, Hing..."


In [34]:
#To create api link for relocating city
filter_field_rel = '[districtname]'
filter_value_rel = Relocating_city

In [35]:
# The code was removed by Watson Studio for sharing.

'https://api.data.gov.in/resource/04cbe4b1-2f2b-4c39-a1d5-1c2e28bc0e32?api-key=579b464db66ec23bdd000001df0247bd5b8a438454e4cd76a7fd7787&filters[districtname]=Mumbai&fields=officename,taluk,districtname&format=csv&offset=0&limit=1000'

In [36]:
# we will call PreProcessUrlToDf that will extract required information from api and create a dataframe 
df_rel_url=PreProcessUrlToDf(api_link_rel)
df_rel_url.head(5)

,Neighborhood,Borough,City
1,"""Antop Hill S.O""",Mumbai,Mumbai
2,"""B P T Colony S.O""",Mumbai,Mumbai
3,"""B.P.Lane S.O""",NA,Mumbai
4,"""BEST STaff Quarters S.O""",NA,Mumbai
5,"""C G S Colony S.O""",Mumbai,Mumbai


In [38]:
#we will call FormatNeighName that will remove unwanted charcter from neighbor hood as seen above like S.O" or B.O"

df_prepros_Neigh_rel=FormatNeighName(df_rel_url)
df_prepros_Neigh_rel.head(5)

,Neighborhood,Borough,City
1,Antop Hill,Mumbai,Mumbai
2,B P T Colony,Mumbai,Mumbai
3,B.P.Lane,NA,Mumbai
4,BEST STaff Quarters,NA,Mumbai
5,C G S Colony,Mumbai,Mumbai


In [41]:
#This will merge all the neighborhood and taluka names that belongs to same pincode. 
df_neighborhood_rel=MergeNeighByTaluka(df_prepros_Neigh_rel)
print(df_neighborhood_rel.shape)
df_neighborhood_rel.head()

(17, 3)


,Borough,City,Neighborhood
0,Bandra,Mumbai,J.B. Nagar
1,Borivali East,Mumbai,"Cumballa Sea Face, Charni Road, Chinchpokli, ..."
2,Borivali West,Mumbai,"Dharavi Road, Holiday Camp, Grant Road"
3,Borivlai East,Mumbai,Kalbadevi H.O
4,Dahisar East,Mumbai,"Chaupati, Bhawani Shankar"


In [40]:
df_neighborhood_all=df_neighborhood_rel.append(df_neighborhood_grouped,ignore_index=True)
df_neighborhood_all.shape

(37, 3)

In [47]:
address = 'Borivali West,Mumbai'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New Delhi are 19.2294561, 72.8479905.


In [49]:
df_neighborhood_loc=ExtractLatLong(df_neighborhood_all)
df_neighborhood_final=df_neighborhood_loc[df_neighborhood_loc.Latitude !='NA',]
df_neighborhood_final.reset_index(drop=True, inplace=True)

In [51]:
df_neighborhood_final=df_neighborhood_final[df_neighborhood_final.Borough != 'NA']
df_neighborhood_final.reset_index(drop=True, inplace=True)
df_neighborhood_final

,Borough,City,Neighborhood,Latitude,Longitude
0,Bandra,Mumbai,High Court Building (Mumbai),19.055,72.8402
1,Borivali East,Mumbai,"Dadar Colony, Santacruz(East), Wadala Rs, C G...",19.2267,72.8619
2,Borivali West,Mumbai,"Kidwai Nagar (Mumbai), IRLA, J.B. Nagar",19.2295,72.848
3,Dahisar East,Mumbai,"Khar Delivery, H.M.P. School",19.2487,72.8641
4,Dahisar West,Mumbai,Khar Colony,19.2484,72.8525
5,Goregaon East,Mumbai,"Netajinagar, Chakala Midc, Kherwadi, B P T Co...",19.1693,72.8553
6,Goregaon West,Mumbai,"Kalachowki, Marol Naka, Marol Bazar, Hanuman ...",19.1633,72.8412
7,Jogeshwari East,Mumbai,"Chamarbaug, Sahar P & T Colony",19.137,72.8649
8,Jogeshwari West,Mumbai,"Sahargaon, Dadar H.O",19.1364,72.8374
9,Kandivali East,Mumbai,Danda,19.2102,72.8649


#### we will use four square API to extract venue details of each speciifed address.Baesd on the venue information we will cluster the relocating city

In [52]:
# The code was removed by Watson Studio for sharing.

In [58]:
# defining a function to extarct venue details for given lattitude and longitude
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])   
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
city_venues = getNearbyVenues( names=df_neighborhood_final['Borough'],
                                   latitudes=df_neighborhood_final['Latitude'],
                                   longitudes=df_neighborhood_final['Longitude']
                                  )
city_venues.shape

Bandra
Borivali East
Borivali West
Dahisar East
Dahisar West
Goregaon East
Goregaon West
Jogeshwari East
Jogeshwari West
Kandivali East
Kandivali West
Malad East
Malad West
Mumbai
Ambegaon
Baramati
Bhor
Daund
Indapur
Junnar
Khed
Mawal
Mulshi
Pune
Pune City
Purandar
Purandhar
Shirur
Velhe


(1683, 7)

In [61]:
city_venues.groupby('Borough').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Ambegaon,1,1,1,1,1,1
Bandra,100,100,100,100,100,100
Baramati,5,5,5,5,5,5
Bhor,4,4,4,4,4,4
Borivali East,100,100,100,100,100,100
Borivali West,100,100,100,100,100,100
Dahisar East,100,100,100,100,100,100
Dahisar West,100,100,100,100,100,100
Daund,4,4,4,4,4,4


In [64]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot=city_onehot.assign(Borough=(city_venues['Borough']))

# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()


,Borough,Airport,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,...,Toll Booth,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Water Park,Women's Store
0,Bandra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bandra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bandra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bandra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bandra,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
city_grouped = city_onehot.groupby('Borough').mean().reset_index()
city_grouped

,Borough,Airport,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,...,Toll Booth,Toll Plaza,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Water Park,Women's Store
0,Ambegaon,1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00
1,Bandra,0.0,0.02,0.00,0.02,0.00,0.00,0.01,0.040000,0.04,...,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.01,0.00,0.01
2,Baramati,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00
3,Bhor,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00
4,Borivali East,0.0,0.00,0.00,0.00,0.00,0.01,0.00,0.010000,0.03,...,0.0,0.0,0.00,0.00,0.000,0.01,0.00,0.00,0.02,0.00
5,Borivali West,0.0,0.00,0.00,0.00,0.00,0.01,0.00,0.010000,0.02,...,0.0,0.0,0.00,0.00,0.000,0.01,0.00,0.00,0.02,0.00
6,Dahisar East,0.0,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,0.02,...,0.0,0.0,0.00,0.00,0.000,0.01,0.00,0.01,0.02,0.00
7,Dahisar West,0.0,0.00,0.00,0.00,0.00,0.01,0.00,0.000000,0.02,...,0.0,0.0,0.00,0.00,0.000,0.01,0.00,0.01,0.02,0.00
8,Daund,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.000,0.25,0.00,0.00,0.00,0.00
9,Goregaon East,0.0,0.01,0.00,0.00,0.00,0.01,0.00,0.030000,0.04,...,0.0,0.0,0.00,0.01,0.000,0.00,0.00,0.00,0.00,0.00


In [67]:
num_top_venues = 5

for hood in city_grouped['Borough']:
    print("----"+hood+"----")
    temp = city_grouped[city_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ambegaon----
           venue  freq
0        Airport   1.0
1  Moving Target   0.0
2            Pub   0.0
3          Plaza   0.0
4     Playground   0.0


----Bandra----
                venue  freq
0   Indian Restaurant  0.10
1         Coffee Shop  0.05
2                Café  0.05
3  Seafood Restaurant  0.04
4       Deli / Bodega  0.04


----Baramati----
               venue  freq
0              Hotel   0.4
1        Bus Station   0.2
2  Indian Restaurant   0.2
3        Pizza Place   0.2
4            Airport   0.0


----Bhor----
        venue  freq
0  Campground  0.25
1       River  0.25
2        Lake  0.25
3        Farm  0.25
4        Park  0.00


----Borivali East----
                  venue  freq
0     Indian Restaurant  0.09
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.05
3            Restaurant  0.04
4        Clothing Store  0.04


----Borivali West----
                  venue  freq
0     Indian Restaurant  0.09
1           Coffee Shop  0.07
2  Fast Food Restaurant  0

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = city_grouped['Borough']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ambegaon,Airport,Gaming Cafe,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market
1,Bandra,Indian Restaurant,Café,Coffee Shop,Deli / Bodega,Seafood Restaurant,Bakery,Bar,Dessert Shop,Scenic Lookout,Italian Restaurant
2,Baramati,Hotel,Indian Restaurant,Pizza Place,Bus Station,Hookah Bar,History Museum,Diner,Donut Shop,Ice Cream Shop,Electronics Store
3,Bhor,Campground,River,Lake,Farm,Women's Store,Farmers Market,Food Truck,Food,Field,Fast Food Restaurant
4,Borivali East,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Clothing Store,Ice Cream Shop,Multiplex,Chinese Restaurant,Scenic Lookout,Pizza Place
5,Borivali West,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Clothing Store,Ice Cream Shop,Café,Chinese Restaurant,Pizza Place,Dessert Shop
6,Dahisar East,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Clothing Store,Restaurant,Scenic Lookout,Ice Cream Shop,Pub,Pizza Place,Café
7,Dahisar West,Indian Restaurant,Fast Food Restaurant,Coffee Shop,Clothing Store,Ice Cream Shop,Restaurant,Shopping Mall,Pub,Multiplex,Café
8,Daund,Platform,Train Station,Diner,Women's Store,Farm,Food Truck,Food,Field,Fast Food Restaurant,Farmers Market
9,Goregaon East,Indian Restaurant,Coffee Shop,Café,Multiplex,Bar,Chinese Restaurant,Italian Restaurant,Hotel,Pizza Place,Ice Cream Shop


In [70]:
# set number of clusters
kclusters = 5

City_grouped_clustering = city_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(City_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0], dtype=int32)

In [71]:
neighborhood_merged = df_neighborhood_final

# add clustering labels
neighborhood_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

neighborhood_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bandra,Mumbai,High Court Building (Mumbai),19.055,72.8402,2,Indian Restaurant,Café,Coffee Shop,Deli / Bodega,Seafood Restaurant,Bakery,Bar,Dessert Shop,Scenic Lookout,Italian Restaurant
1,Borivali East,Mumbai,"Dadar Colony, Santacruz(East), Wadala Rs, C G...",19.2267,72.8619,1,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Clothing Store,Ice Cream Shop,Multiplex,Chinese Restaurant,Scenic Lookout,Pizza Place
2,Borivali West,Mumbai,"Kidwai Nagar (Mumbai), IRLA, J.B. Nagar",19.2295,72.848,1,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Restaurant,Clothing Store,Ice Cream Shop,Café,Chinese Restaurant,Pizza Place,Dessert Shop
3,Dahisar East,Mumbai,"Khar Delivery, H.M.P. School",19.2487,72.8641,1,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Clothing Store,Restaurant,Scenic Lookout,Ice Cream Shop,Pub,Pizza Place,Café
4,Dahisar West,Mumbai,Khar Colony,19.2484,72.8525,1,Indian Restaurant,Fast Food Restaurant,Coffee Shop,Clothing Store,Ice Cream Shop,Restaurant,Shopping Mall,Pub,Multiplex,Café


In [72]:
address = 'Panvel, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New Delhi are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New Delhi are 18.9901717, 73.1215372.


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters